In [1]:
import torch

In [2]:
total_amount = 30_000_000
min_quorum = 2
# min_payout_per_project = 1500
min_payout_per_project = 3_000_000

# NOTE: votes is a matrix
# projects: 0, 1, 2, 3, 4
# voter 0:  3, 1, 5, 0, 0
# voter 1:  2, 0, 8, 3, 1
# voter 2:  1, 1, 2, 4, 0
votes = torch.tensor([
    [3, 1, 5, 0, 0],
    [2, 0, 8, 3, 1],
    [1, 1, 2, 4, 0],
])

1. Filter out projects that receives less than `min_quorum` votes.

In [3]:
project_voters = torch.sum(votes > 0, dim=0)
projects_meet_quorum = project_voters >= min_quorum
filtered_votes = votes.clone()
filtered_votes[:, ~projects_meet_quorum] = 0
print(f"{project_voters=}")
print(f"{projects_meet_quorum=}")
print(f"{filtered_votes=}")

project_voters=tensor([3, 2, 3, 2, 1])
projects_meet_quorum=tensor([ True,  True,  True,  True, False])
filtered_votes=tensor([[3, 1, 5, 0, 0],
        [2, 0, 8, 3, 0],
        [1, 1, 2, 4, 0]])


2. Calculate results based on median

In [4]:
medians = torch.median(filtered_votes.float(), dim=0).values
print(f"{medians=}")

medians=tensor([2., 1., 5., 3., 0.])


3. Scale project payouts to sum up to 30m OP

In [5]:
# 3. Scale project payouts to sum up to 30m OP
total_medians = torch.sum(medians)
proportions_1 = medians.float() / total_medians
scaled_payouts_1 = proportions_1 * total_amount
print("Scaled payouts:", scaled_payouts_1)
sum_after_scaling = int(torch.sum(scaled_payouts_1))
assert sum_after_scaling == total_amount

Scaled payouts: tensor([ 5454545.5000,  2727272.7500, 13636364.0000,  8181818.5000,
               0.0000])


4. Filter out all projects which received below `min_payout_per_project` votes

In [6]:
above_payout_threshold = scaled_payouts_1 >= min_payout_per_project
print(f"{above_payout_threshold=}")
payouts = scaled_payouts_1.clone()
payouts[~above_payout_threshold] = 0
print("Payouts:", payouts)

above_payout_threshold=tensor([ True, False,  True,  True, False])
Payouts: tensor([ 5454545.5000,        0.0000, 13636364.0000,  8181818.5000,
               0.0000])


5. Scale project payouts to sum up to 30m OP

In [7]:
total_payout = torch.sum(payouts)
proportions_2 = payouts.float() / total_payout
scaled_payouts_2 = proportions_2 * total_amount
print("Scaled payouts:", scaled_payouts_2)
sum_after_scaling = int(torch.sum(scaled_payouts_2))
assert sum_after_scaling == total_amount

Scaled payouts: tensor([ 6000000.,        0., 15000000.,  9000000.,        0.])


Now, try to run it with `ezkl`

In [11]:
import os
import json
from torch import nn
import numpy as np

from zkstats.core import (
    gen_settings,
    verifier_setup,
    prover_setup,
    prover_gen_proof,
    verifier_verify,
)


cwd = os.getcwd()

output_dir = f"{cwd}/out"
os.makedirs(output_dir, exist_ok=True)
model_onnx_path = f"{output_dir}/model.onnx"
compiled_model_path = f"{output_dir}/model.compiled"
pk_path = f"{output_dir}/model.pk"
vk_path = f"{output_dir}/model.vk"
proof_path = f"{output_dir}/model.pf"
settings_path = f"{output_dir}/settings.json"
srs_path = f"{output_dir}/kzg.srs"
witness_path = f"{output_dir}/witness.json"
comb_data_path = f"{output_dir}/comb_data.json"

# data tensor
# NOTE: Can we pass `data_tensor` as the matrix as above?
# data = [3.0, 1.0, 5.0, 0.0, 0.0, 2.0, 0.0, 8.0, 3.0, 1.0, 1.0, 1.0, 2.0, 4.0, 0.0]
# data_tensor = torch.tensor(data)
# data_tensor_1 = torch.reshape(torch.tensor(data),(1, len(data), 1))
# comb_data = [data_tensor.tolist()]
# with open(comb_data_path, 'w') as f:
#     json.dump(dict(input_data = comb_data), f)
# data_tensor_array = [data_tensor_1]
data =[[3, 1, 5, 0, 0],
    [2, 0, 8, 3, 1],
    [1, 1, 2, 4, 0],
]
# here comb_data is redundant, but for clarity
comb_data = data
with open(comb_data_path, 'w') as f:
    json.dump(dict(input_data = comb_data), f)

# treat as if we have 3 inputs.
data_tensor_array = []
for arr in data:
    data_tensor_array.append(torch.reshape(torch.tensor(arr), (1, *np.array(arr).shape)))

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

    def forward(self, *args):
        # # some expression of tolerance to error in the inference
        # return filtered_votes
        # X is (1, 15, 1)
        # print("x.shape = ", x.shape)
        # # Y is (1, 15)
        # matrix = x.reshape(-1, 5)
        # print("matrix = ", matrix)
        # print("matrix.shape = ", matrix.shape)
        # print("sum X = ", torch.sum(x))
        # sum_matrix = torch.sum(matrix, dim=0)
        # print("sum matrix = ", sum_matrix)

        # votes = matrix
        # 1.
        matrix = torch.cat((args), dim = 0)
        print(matrix)
        project_voters = torch.sum((matrix > 0).double(), dim=0)
        # projects_meet_quorum = project_voters >= min_quorum
        # filtered_votes = votes.clone()
        # filtered_votes[:, ~projects_meet_quorum] = 0
        # print(f"{project_voters=}")
        # print(f"{projects_meet_quorum=}")
        # print(f"{filtered_votes=}")

        # # 2.
        # medians = torch.mean(filtered_votes.float(), dim=0)
        # print(f"{medians=}")

        # # 3. Scale project payouts to sum up to 30m OP
        # total_medians = torch.sum(medians)
        # proportions_1 = medians.float() / total_medians
        # scaled_payouts_1 = proportions_1 * total_amount
        # print("Scaled payouts:", scaled_payouts_1)
        # # 4. Filter out all projects which received below `min_payout_per_project` votes
        # above_payout_threshold = scaled_payouts_1 >= min_payout_per_project
        # print(f"{above_payout_threshold=}")
        # payouts = scaled_payouts_1.clone()
        # payouts[~above_payout_threshold] = 0
        # # 5. Scale project payouts to sum up to 30m OP
        # total_payout = torch.sum(payouts)
        # proportions_2 = payouts.float() / total_payout
        # scaled_payouts_2 = proportions_2 * total_amount

        # print("scaled_payouts_2 = ", scaled_payouts_2)

        return (torch.tensor(1), project_voters)

        # return (torch.tensor(1), torch.mean(X))


# export_onnx
circuit = Model()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
circuit.to(device)
circuit.eval()
input_names = []
dynamic_axes = {}

data_tensor_tuple = ()
for i in range(len(data_tensor_array)):
    data_tensor_tuple += (data_tensor_array[i],)
    input_index = "input"+str(i+1)
    input_names.append(input_index)
    dynamic_axes[input_index] = {0 : 'batch_size'}
    dynamic_axes["output"] = {0 : 'batch_size'}

torch.onnx.export(
    circuit,               # model being run
    data_tensor_tuple,                   # model input (or a tuple for multiple inputs)
    model_onnx_path,            # where to save the model (can be a file or file-like object)
    export_params=True,        # store the trained parameter weights inside the model file
    opset_version=11,          # the ONNX version to export the model to
    do_constant_folding=True,  # whether to execute constant folding for optimization
    input_names = input_names,   # the model's input names
    output_names = ['output'], # the model's output names
    dynamic_axes=dynamic_axes
)


tensor([[3, 1, 5, 0, 0],
        [2, 0, 8, 3, 1],
        [1, 1, 2, 4, 0]])


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_43663/3453129250.py:102: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.tensor(1), project_voters)


In [12]:

scale = "default"
mode = "resources"
gen_settings(comb_data_path, model_onnx_path, scale, mode, settings_path)


==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":7,"scale_rebase_multiplier":10,"lookup_range":[0,16],"logrows":12,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":3936,"total_assignments":80,"total_const_size":0,"model_instance_shapes":[[1],[5]],"model_output_scales":[0,0],"model_input_scales":[0,0,0],"module_sizes":{"kzg":[],"poseidon":[3936,[3]],"elgamal":[0,[0]]},"required_lookups":[{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [13]:
verifier_setup(model_onnx_path, compiled_model_path, settings_path, srs_path, vk_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 0.44379091262817383 seconds


In [14]:

prover_gen_proof(model_onnx_path, comb_data_path, witness_path, compiled_model_path, settings_path, proof_path, pk_path, srs_path)


spawning module 0
spawning module 2


!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 3.0
witness result 2 : 2.0
witness result 3 : 3.0
witness result 4 : 2.0
witness result 5 : 1.0
==== Generating Proof ====
proof:  {'instances': [[[9296357989369544422, 806886128735292534, 17227576304493195516, 1051098193777516077], [3648264528554272805, 9943096230673625608, 14654808276757294795, 2213601918623149515], [17068183099411130464, 16329882580612051180, 1425198730632413878, 1859139276191188365], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [415066004289224689, 11886516471525959549, 3696305541684646538, 3035258219084094862], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [415066004289224689, 11886516471525959549, 3696305541684646538, 3035258219084094862], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [12436184717236109307, 39

In [15]:
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  3
prf instances:  [[[9296357989369544422, 806886128735292534, 17227576304493195516, 1051098193777516077], [3648264528554272805, 9943096230673625608, 14654808276757294795, 2213601918623149515], [17068183099411130464, 16329882580612051180, 1425198730632413878, 1859139276191188365], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [415066004289224689, 11886516471525959549, 3696305541684646538, 3035258219084094862], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [415066004289224689, 11886516471525959549, 3696305541684646538, 3035258219084094862], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287]]]
proof boolean:  1.0
proof result 1 : 3.0
proof result 2 : 2.0
proof result 3 : 3.0
proof result 4 : 2.0
proof result 5 : 1.0
verified
